# Código do TCC de Paulo de Tarso, da EMAp FGV

In [1]:
import nltk
#Se for preciso, executar o comando abaixo
#nltk.download()
import unicodedata
import string
import os
from sklearn.model_selection import train_test_split as tts
from sklearn.feature_extraction.text import CountVectorizer as CV
#Ignorar warnings
import warnings
warnings.filterwarnings('ignore')

## Códigos auxiliares
### Código para remover símbolos das strings

In [2]:
def clean_string(s):
    s = s.replace("\n"," ")
    return ''.join(x for x in unicodedata.normalize('NFKD', s)
                   if x in string.ascii_letters + " ").lower()

## Parâmetros

In [12]:
#Parâmetros
inicio_atas = 104 #Utilizarei as atas a partir de 2005
final_atas = 206

total_atas = final_atas - inicio_atas + 1
atas = range(inicio_atas,final_atas+1)

path = "D:/Users/paulotarsosantos/Documents/Documentos/TCC/Atas/"

## Tokenização dos textos

#### Obter textos dos arquivos

In [13]:
texts = []

#Ir para a pasta com os arquivos
os.chdir(path)

#Obter o texto dos arquivos
for n in atas:
    file = open("COPOM_" + str(n) + ".txt")
    texts.append(clean_string(file.read()))
    file.close()
    
#Voltar uma pasta
os.chdir("..")

#### Vetorizar os textos obtidos

#### Stopwords

In [14]:
sw = [clean_string(word) for word in 
             open("stopwords.txt",encoding="utf-8").read().splitlines()[1:]]

#### Vectorização

In [47]:
vectorizer = CV(stop_words=sw,min_df=50)
arrays = vectorizer.fit_transform(texts).toarray()
#d_arrays = dict(zip(range(inicio_atas,final_atas+1),arrays))

## Obter os valores das variações na meta da Selic em cada reunião do Copom

In [49]:
file = open("Cortes.txt")
cortes = [int(line.split(";")[2]) for line in file.read().splitlines()[1:]]
cortes = [int(n/abs(n)) if n != 0 else 0 for n in cortes]
cortes.reverse()
file.close()

## Métodos de classificação
Me basearei nos modelos contidos na biblioteca Scikit Learn. Os modelos estão em http://scikit-learn.org/stable/supervised_learning.html.

In [60]:
def RunModel(model):
    classifier = model()
    n = 10000
    score = 0
    last_prediction = 0
    for i in range(n):
        #Separação em dados para treino e dados para teste do modelo
        X_train,X_test,y_train,y_test = tts(arrays[:-1],cortes,train_size=0.9)

        classifier.fit(X_train,y_train)

        score += classifier.score(X_test,y_test)
        last_prediction += classifier.predict(arrays[-1])[0]
    score /= n
    last_prediction /= n
    print("Score:\n" + str(score) + 
          "\nPredição da última reunião:\n" + str(last_prediction))

### GLM

In [61]:
from sklearn.linear_model import LinearRegression as LR
RunModel(LR)

Score:
0.541975385459
Predição da última reunião:
-0.861548588993


### Linear Discriminant Analysis

In [62]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
RunModel(LDA)

Score:
0.762672727273
Predição da última reunião:
-0.9998


### Quadratic Discriminant Analysis

In [63]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
RunModel(QDA)

Score:
0.394236363636
Predição da última reunião:
-0.5381
